In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

from fast_soft_sort.pytorch_ops import soft_rank

# from help_spearman_proxy import *

### MSE for SLR model

In [42]:
y = torch.tensor([1., 2., 3.])

x = torch.tensor([3., 4., 5.], requires_grad=True)
a = torch.tensor(-1.1, requires_grad=True)
b = torch.tensor(5.7, requires_grad=True)
y_hat = a * x + b
# y.requires_grad = True
l = (y - y_hat).norm()
l

tensor(3.2078, grad_fn=<LinalgVectorNormBackward0>)

In [41]:
-((y - y_hat) * x).sum() / l

tensor(-3.9279, grad_fn=<DivBackward0>)

In [43]:
grad = torch.autograd.grad(l, a)
grad

(tensor(-3.9279),)

### MSE for soft ranks and SLR model

In [72]:
rs = 1.0

y = torch.tensor([[1., 2., 3.]], requires_grad=True)
a = torch.tensor(-1., requires_grad=True)
y_hat = a * y

l = (soft_rank(y, regularization_strength=rs) - soft_rank(y_hat, regularization_strength=rs)).norm()
l

tensor(2.8284, grad_fn=<LinalgVectorNormBackward0>)

In [73]:
torch.autograd.grad(l, y)[0]#.item()

tensor([[-1.4142,  0.0000,  1.4142]])

In [60]:
np.sqrt(2)

1.4142135623730951

In [59]:
1/np.sqrt(8)

0.35355339059327373

In [121]:
rs = 1.

t = torch.tensor([[1.2, 2.]], requires_grad=True)
soft_rank(t, regularization_strength=rs)

l = soft_rank(t, regularization_strength=rs)[0, 0]

grad = torch.autograd.grad(l, t, create_graph=True)
torch.autograd.grad(grad[0], t)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [105]:
soft_rank(torch.tensor([[1.2, 2.]]), regularization_strength=rs)

tensor([[1.1000, 1.9000]])

In [117]:
x = torch.tensor([1.1], requires_grad=True)
a = torch.tensor([0.7], requires_grad=True)
y = a * x**2
grad = torch.autograd.grad(y, x, create_graph=True)
hess = torch.autograd.grad(grad[0], x)
hess

(tensor([1.4000]),)

In [122]:
rs = 1.

y = torch.tensor([[2., 1.]])
y_hat = torch.tensor([[1.2, 2.]], requires_grad=True)

def loss(y_hat):
  r = soft_rank(y, regularization_strength=rs)
  r_hat = soft_rank(y_hat, regularization_strength=rs)
  return (r - r_hat).norm()

grad = torch.autograd.functional.jacobian(loss, y_hat)
hess = torch.autograd.functional.hessian(loss, y_hat)

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.